In [78]:
import numpy as np
from typing import List
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os 
import netCDF4 as nc
from sklearn.model_selection import train_test_split

Create a X Design matrix made in shape (400*600, 24)
when each row is a grid point and each feature is a model.

In [79]:
import numpy as np
from typing import List

def flatten_models_to_grid_matrix(interpolated_precip_list: List[np.ndarray]) -> np.ndarray:
    """
    Create a design matrix X where each row represents a grid point, and each column 
    (feature) represents the flattened precipitation data from a different model.

    Parameters:
    -----------
    interpolated_precip_list : List[np.ndarray]
        A list of 2D arrays (matrices) with dimensions (400, 600) representing 
        precipitation data from different models. Each matrix corresponds to a different 
        model.

    Returns:
    --------
    np.ndarray
        A 2D array (design matrix X) of shape (240000, num_models), where:
        - Each row corresponds to a specific grid point across all models.
        - Each column corresponds to the precipitation data for that grid point from 
          a particular model.
    """
    
    # Number of models and number of grid points
    num_models = len(interpolated_precip_list)
    num_grid_points = 400 * 600

    # Initialize an empty matrix to store the reshaped data
    X = np.zeros((num_grid_points, num_models))

    # Loop over each model's matrix and reshape it into a vector
    for i, precip_matrix in enumerate(interpolated_precip_list):
        # Reshape each (400, 600) matrix to (240000,) and assign it to the ith column
        X[:, i] = precip_matrix.reshape(-1)

    return X


In [80]:
data_relative_path = os.path.join('..', 'Data')
ERA5_path = os.path.join(data_relative_path,'ERA5')
CHIRPS_path = os.path.join(data_relative_path,'CHIRPS2')


In [81]:
# Function to interpolate to target dimensions
def extrarpolate_to_target_dim(data, target_dim):
    m, n = data.shape[1:]  # Original dimensions
    interpolated_data = np.empty((data.shape[0], target_dim[0], target_dim[1]))  # Create an empty array to hold the results

    for i in range(data.shape[0]):  # Iterate over the time dimension
        interpolator = scipy.interpolate.RegularGridInterpolator(
            (np.linspace(0, 1, m), np.linspace(0, 1, n)), 
            data[i, :, :], 
            method='linear'
        )
        new_x = np.linspace(0, 1, target_dim[0])
        new_y = np.linspace(0, 1, target_dim[1])
        new_grid_x, new_grid_y = np.meshgrid(new_x, new_y, indexing='ij')
        points = np.stack([new_grid_x.ravel(), new_grid_y.ravel()], axis=-1)
        interpolated_slice = interpolator(points).reshape(target_dim)
        interpolated_data[i, :, :] = interpolated_slice
    return interpolated_data





In [82]:
import netCDF4 as nc
import numpy as np
import os
import scipy.ndimage

data_by_filename = {}
# Define the bounds for latitude and longitude
bounds_lat = [20, 40]
bounds_lon = [20, 50]

# Define the paths where the NetCDF files are saved
files = {
    'MSLP_ERA5_October.nc': 'msl',  # 'msl' is the variable name for Mean Sea Level Pressure
    'GPH_ERA5_October.nc': 'z'      # 'z' is the variable name for Geopotential Height
}

# Function to find the indices that match the latitude and longitude bounds
def find_lat_lon_indices(latitude, longitude, bounds_lat, bounds_lon):
    lat_indices = np.where((latitude >= bounds_lat[0]) & (latitude <= bounds_lat[1]))[0]
    lon_indices = np.where((longitude >= bounds_lon[0]) & (longitude <= bounds_lon[1]))[0]
    return lat_indices, lon_indices

# Function to interpolate the data to target dimensions
def extrapolate_to_target_dim(data, target_shape):
    zoom_factors = [target_shape[i] / data.shape[i] for i in range(len(target_shape))]
    return scipy.ndimage.zoom(data, zoom_factors, order=1)

# Loop through the files and process each one
for file_name, variable in files.items():
    file_path = os.path.join(ERA5_path, file_name)
    
    try:
        # Load the NetCDF file using netCDF4
        dataset = nc.Dataset(file_path, mode='r')

        # Access the variables
        longitude = dataset.variables['longitude'][:]
        latitude = dataset.variables['latitude'][:]
        time = dataset.variables['time'][:]  # Time steps
        data = dataset.variables[variable][:]   # Get the relevant variable (msl or z)

        # Find the indices for the lat/lon bounds
        lat_indices, lon_indices = find_lat_lon_indices(latitude, longitude, bounds_lat, bounds_lon)

        # Subset the data for the defined bounds
        data_subset = data[:, lat_indices, :][:, :, lon_indices]  # Subsetting for lat/lon

        # Interpolate the subsetted data to the target shape (400, 600)
        target_shape = (data_subset.shape[0], 400, 600)  # Keep time, change lat/lon
        data_interpolated = extrapolate_to_target_dim(data_subset, target_shape)
        data_by_filename[file_name] = data_interpolated
        # Check the shape of the interpolated data
        print(f"{file_name} shape for subset (time, lat, lon): {data_interpolated.shape}")
        
        # Optional: Print a specific time step for the subsetted region
        print(f"{file_name} data for first time step: {data_interpolated[0, :, :]}")

        # Close the dataset after extracting data
        dataset.close()

    except FileNotFoundError:
        print(f"File not found at: {file_path}")
    except Exception as e:
        print(f"Error loading the file: {e}")


MSLP_ERA5_October.nc shape for subset (time, lat, lon): (42, 400, 600)
MSLP_ERA5_October.nc data for first time step: [[101699.42311507 101706.21204295 101713.00097082 ... 102028.60665948
  102028.45460181 102028.30254414]
 [101693.68485647 101700.47378435 101707.26271223 ... 102026.24735567
  102025.95899981 102025.67064394]
 [101687.94659787 101694.73552575 101701.52445363 ... 102023.88805186
  102023.4633978  102023.03874374]
 ...
 [101400.26121866 101396.79204851 101393.32287837 ... 101220.3552735
  101219.16387484 101217.97247619]
 [101394.97951417 101391.60360069 101388.2276872  ... 101220.16550218
  101218.97051673 101217.77553127]
 [101389.69780969 101386.41515287 101383.13249604 ... 101219.97573085
  101218.77715861 101217.57858636]]
GPH_ERA5_October.nc shape for subset (time, lat, lon): (42, 400, 600)
GPH_ERA5_October.nc data for first time step: [[4030.22920353 4400.18566087 4770.14211821 ... -292.71020542
  -293.78722567 -294.86424593]
 [3600.31876867 3969.19550645 4338.072

In [83]:
def get_chirps_flat():
    chirps_dataset = nc.Dataset(f"{CHIRPS_path}/chirps_octobers_middle_east_1981_2010.nc") # CHIRPS_Monthly_precipitation
    chirps_precip_data = np.array(chirps_dataset.variables['precip'][:])
    chirps_2D = np.mean(chirps_precip_data,axis=0)
    flat_chirps_2D= chirps_2D.reshape(-1)
    return flat_chirps_2D

In [84]:
def get_precipitation_models() -> list[np.ndarray]:
    """
    Retrieves and processes precipitation models from a specified directory.
    Returns:
        list[np.ndarray]: A list of NumPy arrays where each array represents the mean
                          precipitation data computed across the first axis of the input arrays.
    """
    models: list[np.ndarray] = []
    for file in os.listdir(interpulated_cmip_path):
        model_path = os.path.join(interpulated_cmip_path, file)
        model_data = np.load(model_path)
        model_mean_data = np.mean(model_data, axis=0)
        models.append(model_mean_data)
    return models


# Flatening the Data

In [85]:
ghp_flat_averaged = np.mean(data_by_filename['GPH_ERA5_October.nc'],axis=0) 
ghp_flat = ghp_flat_averaged.reshape(-1)

mslp_flat_averaged = np.mean(data_by_filename['MSLP_ERA5_October.nc'],axis=0) 
mslp_flat = mslp_flat_averaged.reshape(-1)

y_chirps : np.ndarray = get_chirps_flat()
y_chirps = y_chirps.astype(np.float64)

# Flattening the data 
# X : np.ndarray = matrix with the features 
# sea-level pressure | temp | geo-potetntial pressure 
# Predicted preciptations (the observed ones)




# Split The Data randomly 

In [86]:
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming `ghp_flat` and `mslp_flat` are already calculated and reshaped to 1D arrays
# Combine `ghp_flat` and `mslp_flat` to form X (features)
X = np.column_stack((ghp_flat, mslp_flat))

# Assuming `y_chirps` is the target variable and already in the correct format
# Verify that the lengths match before proceeding
assert len(X) == len(y_chirps), "The lengths of X and y_chirps do not match!"

# Split the data into training and testing sets (80% train, 20% test by default)
X_train, X_test, y_train, y_test = train_test_split(X, y_chirps, test_size=0.2, random_state=42)

# Check the shapes of the splits
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")


X_train shape: (192000, 2), X_test shape: (48000, 2)
y_train shape: (192000,), y_test shape: (48000,)


# Trainig Linear Regression models 

In [87]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Create a Linear Regression model
linear_model = LinearRegression()

# Fit the model on the training data
linear_model.fit(X_train, y_train)

# Predict on the test data
y_pred_linear = linear_model.predict(X_test)

# Calculate MSE and R² on test data
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

# Display the results
print(f"Linear Regression MSE: {mse_linear}")
print(f"Linear Regression R²: {r2_linear}")
print(f"Linear Regression Coefficients: {linear_model.coef_}")
print(f"Linear Regression Intercept: {linear_model.intercept_}")


Linear Regression MSE: 1.406319733890696e+73
Linear Regression R²: 0.010306664613478622
Linear Regression Coefficients: [-7.44006046e+31  1.07062795e+33]
Linear Regression Intercept: -1.0655355112613028e+38


In [88]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Create polynomial features (degree=2 for quadratic, change degree for higher-order polynomials)
poly = PolynomialFeatures(degree=2)

# Transform the features to polynomial features
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Create a Linear Regression model (for polynomial regression)
poly_model = LinearRegression()

# Fit the model on the polynomial features of the training data
poly_model.fit(X_train_poly, y_train)

# Predict on the polynomial features of the test data
y_pred_poly = poly_model.predict(X_test_poly)

# Calculate MSE and R² on test data
mse_poly = mean_squared_error(y_test, y_pred_poly)
r2_poly = r2_score(y_test, y_pred_poly)

# Display the results
print(f"Polynomial Regression (degree 2) MSE: {mse_poly}")
print(f"Polynomial Regression (degree 2) R²: {r2_poly}")
print(f"Polynomial Regression Coefficients: {poly_model.coef_}")
print(f"Polynomial Regression Intercept: {poly_model.intercept_}")


Polynomial Regression (degree 2) MSE: 1.2699815282289448e+73
Polynomial Regression (degree 2) R²: 0.1062542718682592
Polynomial Regression Coefficients: [ 0.00000000e+00 -1.37708579e+35  3.53940900e+36 -3.74530326e+28
  1.36000399e+30 -1.74593213e+31]
Polynomial Regression Intercept: -1.793773360681875e+41


# coefficient evaluetion

In [89]:
def plot_coefficients(coefficients: np.ndarray, model_names: List[str], title: str):
    """
    Plot the coefficients of the regression model.

    Parameters:
    -----------
    coefficients : np.ndarray
        The regression coefficients for each model.
        
    model_names : List[str]
        The names or labels of the models corresponding to the coefficients.

    title : str
        Title of the plot.
    """
    plt.figure(figsize=(10, 6))
    plt.barh(model_names, coefficients, color='skyblue')
    plt.xlabel('Coefficient Value')
    plt.title(title)
    plt.show()